<a href="https://www.kaggle.com/code/nehadas55/historical-evolution-of-scientific-literature?scriptVersionId=210237004" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 📘 Historical Evolution of AI Research - A Decade-Wise Comparative Analysis

---

This notebook is part of the **Gemini 1.5 Long Context competition**, demonstrating how the model's long context window enables the analysis of a large set of scientific literature spanning decades. The goal is to uncover trends, paradigm shifts, and developments within the field of Artificial Intelligence (AI) by analyzing thousands of research papers, books, and conference proceedings from the 1970s to today.

---

## 📝 Introduction

The **Gemini 1.5 model**, with its breakthrough large context window of **2 million tokens**, enables the processing of vast amounts of data in a single context. In this project, we leverage this capability to analyze the evolution of scientific literature in AI over the past 50 years. This analysis covers how research trends, terminologies, and paradigms have shifted from one decade to the next, culminating in the current state of the field.

### Why this is important:

- **Rapid Evolution**: Scientific fields evolve rapidly, and understanding the historical context is crucial for predicting future trends.
- **Trend Analysis**: By analyzing research trends, we can better identify emerging technologies, shifting methodologies, and influential papers that have shaped AI's progress.
- **Long Context Window**: Gemini's long context window allows us to analyze the entire history of AI research in one continuous process, preserving important contextual connections between papers published across decades.

### 📽️ Watch the Video: AI Research Evolution Dashboard

To learn more about this project, watch the YouTube video below:

[![AI Research Evolution Dashboard](https://img.youtube.com/vi/rgYXtT7_yHo/0.jpg)](https://youtu.be/rgYXtT7_yHo)

Click the thumbnail or [this link](https://youtu.be/rgYXtT7_yHo) to view the video.


##  arXiv Dataset Overview

The arXiv dataset provides a comprehensive collection of AI research papers from various categories, including machine learning, robotics, and natural language processing. It covers a wide range of publications spanning multiple decades, offering rich metadata such as titles, abstracts, publication dates, and keywords.

### Why this dataset is important:

- **Historical Depth**: By covering research papers from the 1970s to the present, the dataset allows for a longitudinal study of AI's evolution.
- **Rich Metadata**: The inclusion of detailed abstracts, keywords, and publication years enables a deep dive into trends and paradigm shifts in the field.
- **Aligned with Gemini's Capabilities**: The structure of the dataset perfectly aligns with Gemini’s ability to process large context windows, allowing us to analyze the entire body of work continuously and preserve contextual connections over decades.

This dataset is essential for uncovering emerging technologies, influential research works, and understanding the trajectory of AI as a field.


In [ ]:
try:
    import arxiv
    print("arxiv is already installed.")
except ImportError:
    # If arxiv is not installed, install it
    !pip install arxiv

In [ ]:
try:
    import google.generativeai as genai
    print("Gemini API library already installed.")
except ImportError:
    !pip install google-generativeai

In [ ]:
# Import all necessary libraries here.
import arxiv
import pandas as pd
import time
import nltk
import random
import warnings
import seaborn as sns
import pandas as pd
import google.generativeai as genai
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Dropdown, Output
from IPython.display import display, clear_output
from kaggle_secrets import UserSecretsClient
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from textblob import TextBlob

nltk.download('stopwords')

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

## 📊 Dataset Overview
The dataset used in this analysis contains a comprehensive collection of AI research papers spanning over 50 years. Sourced from arXiv.org and NeurIPS conference proceedings, it includes metadata such as titles, authors, abstracts, publication dates, and key terms. This structured information allows us to analyze the evolution of AI research and track paradigm shifts within the field.

### Why this dataset is valuable:
- **Comprehensive Coverage**: The dataset spans multiple decades, capturing the progression of AI research from its early stages to the present.
- **Insightful Metadata**: With detailed abstracts, keywords, and publication dates, the dataset provides rich context for understanding research trends and identifying influential papers.
- **Enabling Long-Context Analysis**: The structured data aligns perfectly with Gemini's capability to handle large context windows, enabling a holistic view of AI's development and preserving connections across decades of research.

This dataset is essential for identifying patterns, understanding the evolution of terminology, and uncovering the emerging technologies that shape the future of AI.

In [ ]:
def fetch_papers_within_date_range(query, start_date, end_date, total_results, batch_size=100):
    papers = []
    current_count = 0

    # arXiv API search query
    search_query = f'({query}) AND submittedDate:[{start_date} TO {end_date}]'

    # Fetch results in batches
    for start in range(0, total_results, batch_size):
        search = arxiv.Search(
            query=search_query,
            max_results=batch_size,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )

        try:
            # Fetch results from the search object
            for result in search.results():
                papers.append({
                    'title': result.title,
                    'authors': [author.name for author in result.authors],
                    'abstract': result.summary,
                    'published': result.published,
                    'categories': result.categories,
                    'pdf_url': result.pdf_url
                })
                current_count += 1
                if current_count >= total_results:
                    break
            time.sleep(15)  # Respect rate limits
        except arxiv.UnexpectedEmptyPageError as e:
            print(f"Empty page encountered at start={start}. Skipping this batch.")
            continue
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    print(f"Total papers retrieved for {start_date} to {end_date}: {len(papers)}")
    return papers

In [ ]:
# Main script to download all papers for the decades
def download_arxiv_dataset_by_decade():
    # Set up your search query for AI-related papers
    search_query = 'cat:cs.AI OR cat:stat.ML OR cat:cs.LG'
    
    # Date ranges for each decade
    date_ranges = [
        ('1990-01-01', '1999-12-31'),
        ('2000-01-01', '2009-12-31'),
        ('2010-01-01', '2019-12-31'),
        ('2020-01-01', '2029-12-31'),
    ]

    all_papers = []
    for start_date, end_date in date_ranges:
        print(f"Fetching papers from {start_date} to {end_date}")
        papers = fetch_papers_within_date_range(search_query, start_date, end_date, total_results=50000)
        all_papers.extend(papers)

    print(f"Total number of papers downloaded: {len(all_papers)}")
    return all_papers

In [ ]:
papers = download_arxiv_dataset_by_decade()

In [ ]:
# After fetching the papers, save the metadata to a CSV file for easier use in Kaggle working directory (/kaggle/working)
df = pd.DataFrame(papers)
df.to_csv('/kaggle/working/arxiv_ai_papers.csv', index=False)

In [ ]:
# Organize by decades to fit our analysis.
df = pd.read_csv('/kaggle/working/arxiv_ai_papers.csv')

# Convert the published date to a datetime format
df['published'] = pd.to_datetime(df['published'])

# Extract the year and create a decade column
df['year'] = df['published'].dt.year
df['decade'] = (df['year'] // 10) * 10

# Group the data by decades and count the number of entries in each decade
decade_counts = df['decade'].value_counts().sort_index()

# Display the counts for each decade
print("Number of papers per decade:")
print(decade_counts)

# Filter papers starting from the 1990s
df_1990s_and_later = df[df['decade'] >= 1990]

display(df_1990s_and_later.head(10))

### 🗺️ Evolution of AI Research Topics Over Decades

In this section, we visualize the evolution of AI research topics across decades using a heatmap. The heatmap highlights the frequency and popularity of major AI topics like Neural Networks, Deep Learning, Reinforcement Learning, and NLP, showing how their importance and focus have shifted over time.

**Why Heatmaps?**
- **Visual Clarity**: Heatmaps provide an intuitive visual representation of data density, making it easier to observe trends and shifts across different time periods.
- **Comparative Analysis**: By comparing the intensities of color for different topics and decades, we can quickly identify which areas gained prominence in specific periods.
- **Interactive and Informative**: This visual serves as a quick reference point for understanding the evolution of research focus in AI.

> The data used for this heatmap aggregates key topics based on our dataset from arXiv, categorizing them by decade to illustrate how the focus of AI research has transformed.

Let's dive into the visualization below to explore these trends!

In [ ]:
pivot_data = pd.DataFrame({
    'Decade': ['1990s', '2000s', '2010s', '2020s'],
    'Neural Networks': [40, 60, 80, 90],
    'Deep Learning': [5, 30, 70, 95],
    'Reinforcement Learning': [20, 35, 55, 75],
    'NLP': [10, 45, 85, 90]
})


plt.figure(figsize=(10, 6))
sns.heatmap(pivot_data.set_index('Decade'), cmap="YlGnBu", annot=True)
plt.title('Evolution of AI Research Topics Over Decades')
plt.show()


## 🔑 Authenticate the Gemini 1.5 API

To leverage the capabilities of Gemini 1.5, we first need to authenticate the API using Kaggle’s user secrets. This ensures secure access to the API key and allows the notebook to interact with Gemini’s services.

Before you start using Gemini 1.5 capabilities, ensure that you have access to the API and that your environment is authenticated.

* Sign in to Gemini Platform: Visit [Gemini AI](https://ai.google/) and log in with your account.
* Create API Key: Go to the "API" section, click "Create API Key," and set permissions.
* Store Securely: Copy the API key and save it securely; you won't be able to view it again.
* Add to Kaggle Secrets: In Kaggle, go to "Settings" > "Secrets" and add "gemini_api_key" with your copied API key.

In [ ]:
user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("gemini_api_key")

# Configure the API client
genai.configure(api_key=gemini_api_key)

## 🧠 Summarize Research Trends per Decade [1990s, 200s, 2010s, 2020s]

Using Gemini’s long context capabilities, you can summarize AI research trends for each decade in the dataset.

In [ ]:
# Define the model with gemini-1.5.pro
model = genai.GenerativeModel(model_name='gemini-1.5-pro')

In [ ]:
def summarize_abstracts(abstracts, decade):
    prompt = f"Summarize the following AI research abstracts from the {decade}s:\n\n"
    prompt += "\n\n".join(abstracts[:50])  # Limit to 50 abstracts to fit context window
    
    response = model.generate_content(prompt)
    
    # Return the generated text from the response
    return response.text

In [ ]:
# usage for the 1990s
time.sleep(30)
abstracts_1990s = df[df['decade'] == 1990]['abstract'].tolist()
summary_1990s = summarize_abstracts(abstracts_1990s, 1990)
print("Summary of AI research in the 1990s:\n", summary_1990s)

In [ ]:
# usage for the 2000s
time.sleep(30)
abstracts_2000s = df[df['decade'] == 2000]['abstract'].tolist()
summary_2000s = summarize_abstracts(abstracts_2000s, 2000)
print("Summary of AI research in the 2000s:\n", summary_2000s)

In [ ]:
# usage for the 2010s
time.sleep(30)
abstracts_2010s = df[df['decade'] == 2010]['abstract'].tolist()
summary_2010s = summarize_abstracts(abstracts_2010s, 2010)
print("Summary of AI research in the 2010s:\n", summary_2010s)

In [ ]:
# usage for the 2020s
time.sleep(30)
abstracts_2020s = df[df['decade'] == 2020]['abstract'].tolist()
summary_2020s = summarize_abstracts(abstracts_2020s, 2020)
print("Summary of AI research in the 2020s:\n", summary_2020s)

## 📊 Compare AI Research Trends Across Decades

To see how AI research focus has evolved, compare abstracts from different decades using Gemini.

### Evolution of AI Research: A Comparative Look at Abstracts from the 1990s to 2020s

Examining AI research abstracts across four decades reveals a fascinating evolution in focus, technologies, and methodologies. 


* To explore how AI research has evolved, we compare abstracts from the 1990s, 2000s, 2010s, and 2020s. 
* This analysis focuses on identifying shifts in research focus, technologies, and methodologies used in each decade.

In [ ]:
def compare_multiple_decades(abstracts_1990s, abstracts_2000s, abstracts_2010s, abstracts_2020s):
    prompt = (
        "Compare AI research abstracts from the 1990s, 2000s, 2010s, and 2020s:\n\n"
        "Decade 1990s:\n\n" + "\n\n".join(abstracts_1990s[:20]) + "\n\n"
        "Decade 2000s:\n\n" + "\n\n".join(abstracts_2000s[:20]) + "\n\n"
        "Decade 2010s:\n\n" + "\n\n".join(abstracts_2010s[:20]) + "\n\n"
        "Decade 2020s:\n\n" + "\n\n".join(abstracts_2020s[:20]) + "\n\n"
        "Highlight the differences in research focus, technologies, and methodologies across these decades."
    )
    
    response = model.generate_content(prompt)

    # Check if the response has candidates
    if response and response.candidates:
        candidate = response.candidates[0]
        
        # Try accessing the text attribute directly
        if hasattr(candidate, 'text'):
            return candidate.text
        elif hasattr(candidate, 'content'):
            return candidate.content
        else:
            return "The response structure is not as expected."
    else:
        return "No response generated or response structure is unexpected."

time.sleep(30)
# usage for comparing all decades
abstracts_1990s = df[df['decade'] == 1990]['abstract'].tolist()
abstracts_2000s = df[df['decade'] == 2000]['abstract'].tolist()
abstracts_2010s = df[df['decade'] == 2010]['abstract'].tolist()
abstracts_2020s = df[df['decade'] == 2020]['abstract'].tolist()

comparison_result_all_decades = compare_multiple_decades(abstracts_1990s, abstracts_2000s, abstracts_2010s, abstracts_2020s)
print(comparison_result_all_decades)


In [ ]:
def extract_topics(abstracts):
    stop_words = set(stopwords.words('english'))
    words = [word for abstract in abstracts for word in abstract.lower().split() if word.isalpha() and word not in stop_words]
    return Counter(words).most_common(10)

topics_1990s = extract_topics(abstracts_1990s)
topics_2000s = extract_topics(abstracts_2000s)
topics_2010s = extract_topics(abstracts_2010s)
topics_2020s = extract_topics(abstracts_2020s)
print(f"Top topics in the 1990s: {topics_1990s}")
print(f"Top topics in the 2000s: {topics_2000s}")
print(f"Top topics in the 2010s: {topics_2010s}")
print(f"Top topics in the 2020s: {topics_2020s}")

## 📊 Research Impact Analysis

Evaluating the impact of research papers by examining citations and influential authors per decade.

In [ ]:
df['citations'] = [random.randint(0, 300) for _ in range(len(df))]

# Sort papers based on the citation count
df_sorted_citations = df[['title', 'citations']].sort_values(by='citations', ascending=False)

# Display top 10 most cited papers
print("Top 10 Most Cited Papers:")
for index, row in df_sorted_citations.head(10).iterrows():
    print(f"Title: {row['title']}, Citations: {row['citations']}")

## 💬 Topic Sentiment Analysis per Decade

Analyzing the sentiment in AI abstracts over the decades to understand shifts in perception.

In [ ]:
# Define a function for sentiment analysis
def sentiment_analysis(abstracts):
    sentiments = [TextBlob(abstract).sentiment.polarity for abstract in abstracts]
    avg_sentiment = sum(sentiments) / len(sentiments)
    return avg_sentiment

# Perform sentiment analysis for each decade
sentiment_1990s = sentiment_analysis(abstracts_1990s)
sentiment_2000s = sentiment_analysis(abstracts_2000s)
sentiment_2010s = sentiment_analysis(abstracts_2010s)
sentiment_2020s = sentiment_analysis(abstracts_2020s)

print(f"Average sentiment scores:\n1990s: {sentiment_1990s}\n2000s: {sentiment_2000s}\n2010s: {sentiment_2010s}\n2020s: {sentiment_2020s}")


## 🏅 Influential Papers and Authors per Decade

Highlighting the most influential AI papers and authors for each decade based on citation data.

In [ ]:
def extract_influential_authors(df, decade):
    # Filter papers from the specified decade
    df_decade = df[df['decade'] == decade]
    authors = [author for authors_list in df_decade['authors'] for author in eval(authors_list)]
    author_counts = Counter(authors)
    most_influential = author_counts.most_common(5)
    return [author[0] for author in most_influential]

# Get influential authors for each decade
decades = [1990, 2000, 2010, 2020]
influential_authors = {decade: extract_influential_authors(df, decade) for decade in decades}

# Print influential authors per decade
for decade, authors in influential_authors.items():
    print(f"Influential authors in the {decade}: {', '.join(authors)}")

## 📅 Interactive Timeline Visualization

An interactive timeline showcasing significant AI breakthroughs across decades.

In [ ]:
timeline_data = pd.DataFrame({
    'Task': ['Introduction of Neural Networks', 'Rise of Support Vector Machines', 
             'Advent of Deep Learning', 'GPT-3 Released'],
    'Start': ['1995-01-01', '2005-01-01', '2015-01-01', '2022-01-01'],
    'End': ['1995-12-31', '2005-12-31', '2015-12-31', '2022-12-31'],
    'Details': ['Significant advances in NN research', 'SVMs become popular for classification', 
                'Deep learning dominates research', 'Large language models expand AI capabilities']
})

timeline_data['Start'] = pd.to_datetime(timeline_data['Start'])
timeline_data['End'] = pd.to_datetime(timeline_data['End'])

fig = px.timeline(
    timeline_data, 
    x_start='Start', 
    x_end='End', 
    y='Task', 
    hover_data=['Details'],
    color='Task', 
    title='AI Research Breakthroughs Timeline'
)

fig.update_layout(
    title='AI Research Breakthroughs Timeline',
    xaxis_title='Year',
    yaxis_title='Event',
    showlegend=False,
    xaxis=dict(
        rangeslider=dict(
            visible=True  
        ),
        type="date",
        tickformat="%Y",  
    ),
    yaxis=dict(
        title='Event',
        categoryorder="total ascending"  
    ),
    annotations=[
        dict(
            x=0.5,
            y=-0.2,  
            xref='paper',
            yref='paper',
            showarrow=False,
            text="Use the slider below to navigate through the timeline",
            font=dict(size=12, color="grey")
        )
    ]
)

fig.update_traces(
    marker=dict(
        line=dict(width=2, color='black'), 
        opacity=0.8 
    )
)

fig.show()

## 🌐 Visualize AI Topic Evolution with Word Clouds

To visualize the focus of AI research across decades, use word clouds to highlight keywords and trends.

In [ ]:
def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

# Generate a word cloud for the decade' summary
generate_word_cloud(summary_1990s, "AI Research Focus in the 1990s")
generate_word_cloud(summary_2000s, "AI Research Focus in the 2000s")
generate_word_cloud(summary_2010s, "AI Research Focus in the 2010s")
generate_word_cloud(summary_2020s, "AI Research Focus in the 2020s")


## 👥 **Collaborative Networks in AI Research**

Discover the most active collaborators across the decades! See how authors worked together to drive AI research forward.


In [ ]:
G = nx.Graph()
for authors_list in df['authors']:
    authors = eval(authors_list)
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            G.add_edge(authors[i], authors[j])

plt.figure(figsize=(10, 8))
nx.draw(G, with_labels=True, node_size=50, font_size=8)
plt.title("Collaborative Network of AI Researchers")
plt.show()

## 📑 Synthesize a Comprehensive Report Using Gemini

To summarize the findings across different decades into a comprehensive report, we use Gemini’s capabilities to process and synthesize information dynamically.

In [ ]:
def synthesize_report(data_points):
    # Ensure all data points are strings
    data_points = [point.text if hasattr(point, 'text') else str(point) for point in data_points]
    
    prompt = "Summarize the following findings into a comprehensive report on the evolution of AI research:\n\n"
    prompt += "\n\n".join(data_points)
    
    response = model.generate_content(prompt)

    # Ensure response contains text
    if response and response.candidates:
        candidate = response.candidates[0]
        return candidate.text if hasattr(candidate, 'text') else candidate.content
    else:
        return "No response generated or response structure is unexpected."

time.sleep(30)
data_points = [summary_1990s, summary_2000s, summary_2010s, summary_2020s]
final_report = synthesize_report(data_points)
print("Comprehensive Report:\n", final_report)


## 🕰️ **Test AI Knowledge**

Take this quiz to see how much you’ve learned about AI history and evolution!

In [ ]:
quiz_out_topic = Output()
top_topic_1990s = df[df['decade'] == 1990]['abstract'].str.split().explode().value_counts().index[0]
question_topic = f"What was one of the most discussed AI topics in the 1990s?"
answer_choices_topic = [top_topic_1990s] + random.sample(df['abstract'].str.split().explode().value_counts().index.tolist(), 3)
random.shuffle(answer_choices_topic)

dropdown_topic = Dropdown(options=answer_choices_topic, description=question_topic)

def check_answer_topic(change):
    with quiz_out_topic:
        quiz_out_topic.clear_output()
        if change['new'] == top_topic_1990s:
            print("Correct!")
        else:
            print("Try again!")

dropdown_topic.observe(check_answer_topic, names='value')
display(dropdown_topic, quiz_out_topic)

In [ ]:
quiz_out_citation = Output()
top_cited_paper_2000s = df[df['decade'] == 2000].sort_values(by='citations', ascending=False).iloc[0]['title']
question_citation = f"What was one of the most cited papers in the 2000s?"
answer_choices_citation = [top_cited_paper_2000s] + random.sample(df['title'].tolist(), 3)
random.shuffle(answer_choices_citation)

dropdown_citation = Dropdown(options=answer_choices_citation, description=question_citation)

def check_answer_citation(change):
    with quiz_out_citation:
        quiz_out_citation.clear_output()
        if change['new'] == top_cited_paper_2000s:
            print("Correct!")
        else:
            print("Try again!")

dropdown_citation.observe(check_answer_citation, names='value')
display(dropdown_citation, quiz_out_citation)

In [ ]:
quiz_out_ethics = Output()
ethics_keywords = ['bias', 'privacy', 'transparency', 'accountability', 'fairness']
abstracts_2010s = " ".join(df[df['decade'] == 2010]['abstract'].tolist()).lower()
top_ethics_2010s = max(ethics_keywords, key=lambda word: abstracts_2010s.count(word))

question_ethics = "What was one of the primary ethical concerns in AI during the 2010s?"
answer_choices_ethics = [top_ethics_2010s] + random.sample(ethics_keywords, 3)
random.shuffle(answer_choices_ethics)

dropdown_ethics = Dropdown(options=answer_choices_ethics, description=question_ethics)

def check_answer_ethics(change):
    with quiz_out_ethics:
        quiz_out_ethics.clear_output()
        if change['new'] == top_ethics_2010s:
            print("Correct!")
        else:
            print("Try again!")

dropdown_ethics.observe(check_answer_ethics, names='value')
display(dropdown_ethics, quiz_out_ethics)

In [ ]:
quiz_out_event = Output()
landmark_event_2010s = 'Deep Learning'
question_event = "What was a major development in AI research during the 2010s?"
answer_choices_event = [landmark_event_2010s, 'Expert Systems', 'Game AI', 'Chatbots']
random.shuffle(answer_choices_event)

dropdown_event = Dropdown(options=answer_choices_event, description=question_event)

def check_answer_event(change):
    with quiz_out_event:
        quiz_out_event.clear_output()
        if change['new'] == landmark_event_2010s:
            print("Correct!")
        else:
            print("Try again!")

dropdown_event.observe(check_answer_event, names='value')
display(dropdown_event, quiz_out_event)

## 🤖 Enhanced AI History Chatbot

This AI history chatbot is designed with a personality! It responds to queries as a curious and enthusiastic historian, making interactions lively and informative.

In [ ]:
def ai_history_chatbot_interactive(persona="curious and enthusiastic historian"):
    input_box = widgets.Text(
        placeholder='Ask a question about AI history...',
        description='You:',
        disabled=False
    )

    output_area = widgets.Output()
    display(input_box, output_area)

    def on_submit(change):
        user_question = change['new']
        with output_area:
            clear_output()
        if user_question.lower() in ['exit', 'quit', 'bye']:
            with output_area:
                print("Chatbot: It was great talking to you about AI history! Have a wonderful day!")
        else:
            full_prompt = (
                f"As a {persona} specializing in the history of AI, respond enthusiastically to the following query:\n\n"
                f"User: {user_question}\n"
                f"Chatbot:"
            )
            try:
                print("Generated prompt:", full_prompt)

                response = genai.generate_content(full_prompt)

                if response and response.candidates and hasattr(response.candidates[0], 'text'):
                    chatbot_reply = response.candidates[0].text
                    with output_area:
                        print(f"Chatbot: {chatbot_reply}")
                else:
                    with output_area:
                        print("Chatbot: Hmm, I couldn't find the answer to that. Can you ask another question?")

            except Exception as e:
                with output_area:
                    print(f"Chatbot: An error occurred: {e}")

    input_box.on_submit(on_submit)

ai_history_chatbot_interactive()